## Libraries

In [ ]:
import pandas                  as pd
import matplotlib.pyplot       as plt
import seaborn                 as sns
import numpy                   as np
from itertools                 import count
from sklearn.metrics.pairwise  import euclidean_distances
from sklearn.neighbors         import KNeighborsClassifier
from sklearn.metrics           import accuracy_score, confusion_matrix
from sklearn.naive_bayes       import GaussianNB
from sklearn.preprocessing     import StandardScaler, Normalizer
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.metrics           import mean_squared_error
from scipy.signal              import savgol_filter
from statsmodels.tsa.holtwinters import SimpleExpSmoothing

#### Understanding the data

##### Read the data

In [ ]:
data = pd.read_csv('data/drought_forecasting.csv', dayfirst=True, parse_dates =["date"], index_col ="date")
data

##### Missing values

In [ ]:
nullValues = data.isnull().sum()
nullValues

##### Data types and data summary

In [ ]:
data.head()

In [ ]:
data.describe()
# looking the data description is already possible to see that aggregate the data could be not so good, 
# because there is a high value in PRECTOT that can be an outliers and will 'desapear' in the aggregation process.
# So use the atomic granularity possibly is the best option

In [ ]:
data.dtypes

In [ ]:
"""data['day'] = pd.DatetimeIndex(data['date'], dayfirst=True).day
data['month'] = pd.DatetimeIndex(data['date'], dayfirst=True).month
data['year'] = pd.DatetimeIndex(data['date'], dayfirst=True).year
data = data[[col for col in data if col not in ['QV2M']] + ['QV2M']]
data.pop('date')"""

#### 1. Data Profiling

##### 1.1. Data Granularity 

###### 1.1.1. Resample

In [ ]:
# Granularity: atomic (daily)
daily_data = data
# Granularity: weekly
weekly_data = data.resample('W').mean()
# Granularity: monthly
monthly_data = data.resample('M').mean()

##### 1.2. Data Distribution and Stationarity  

###### 1.2.1. Boxplots

In [ ]:
def boxplot(data, filename):
    """"""

    #fig, ax = plt.subplots()
    sns.boxplot(data=data)
    plt.savefig('plots/'+filename + ".png")
    plt.close()

In [ ]:
boxplot(daily_data, 'boxplot_drought_forecasting_dailySeparated')
boxplot(weekly_data, 'boxplot_drought_forecasting_weeklySeparated')
boxplot(monthly_data, 'boxplot_drought_forecasting_monthlySeparated')

###### 1.2.2. Histograms 

In [ ]:
def histograms(data, filename, dimension):
    i, j = dimension
    fig, ax = plt.subplots(i, j, figsize=(24, 14))
    
    for position in range(len(data.columns)):
        col = data.columns[position]

        pos_i = position//j
        pos_j = position%j

        ax[pos_i][pos_j].hist(data[col])
        ax[pos_i][pos_j].set_title(col)
        ax[pos_i][pos_j].legend()
        
    plt.savefig('plots/' + filename + '.png')
    plt.close()

In [ ]:
histograms(daily_data, 'hist_drought_forecasting_dailySeparated', (2, 4))
histograms(weekly_data, 'hist_drought_forecasting_weeklySeparated', (2, 4))
histograms(monthly_data, 'hist_drought_forecasting_monthlySeparated', (2, 4))

###### 1.2.3. Stationarity

In [ ]:
def plot_stationarity(data, filename, dimension):
    i, j = dimension
    fig, ax = plt.subplots(i, j, figsize=(24, 14))
    
    for position in range(len(data.columns)):
        col = data.columns[position]

        pos_i = position//j
        pos_j = position%j

        ax[pos_i][pos_j].plot(data[col])
        ax[pos_i][pos_j].set_title(col)
        ax[pos_i][pos_j].legend()
        
    plt.savefig('plots/' + filename + '.png')
    plt.close()
    

In [ ]:
plot_stationarity(daily_data, 'stationarity_drought_forecantig_dailySeparated', (2, 4))
plot_stationarity(weekly_data, 'stationarity_drought_forecantig_weeklySeparated', (2, 4))
plot_stationarity(monthly_data, 'stationarity_drought_forecantig_monthlySeparated', (2, 4))

In [ ]:
def temporal_data_split_forPersistence(data, train_size=0.80):

    data = pd.DataFrame(data)
    lim = round(len(data)*train_size)

    tmp_data_x = data.shift(1)
    X_train = tmp_data_x.iloc[1:lim]
    X_test = tmp_data_x.iloc[lim:]
    Y_train = data.iloc[1:lim]
    Y_test = data.iloc[lim:]
    return X_train, X_test, Y_train, Y_test

In [ ]:
# persistence model
def model_persistence(x):
    return x

In [ ]:
def persistence_model(X_test, Y_test):

   predictions = []
   for x in np.array(X_test):
      yhat = model_persistence(x)
      predictions.append(yhat)
      
   rmse = (mean_squared_error(Y_test, predictions))**(1/2)
   res = [Y_test, pd.DataFrame(predictions, index=X_test.index)]
   return res, rmse

In [ ]:
def plot_persistence(data, columns,  filename, dimension):
    i, j = dimension
    fig, ax = plt.subplots(i, j, figsize=(35, 25))
    
    for position in range(len(columns)):
        col = columns[position]

        pos_i = position//j
        pos_j = position%j

        ax[pos_i][pos_j].plot(data[position][0], color = "red")
        ax[pos_i][pos_j].plot(data[position][1], color = "green")
        plt.legend(col)
        plt.xticks(rotation=45)

    plt.savefig('plots/' + filename + '.png')

In [ ]:
data_daily_persistenceRes = []
data_weekly_persistenceRes = []
data_monthly_persistenceRes = []

stats_ps = pd.DataFrame(columns=['var', 'rmse - daily', 'rmse - weekly', 'rmse - monthly'])

c = 0
for col in daily_data.columns:

    X_train_d, X_test_d, Y_train_d, Y_test_d = temporal_data_split_forPersistence(daily_data[col])
    X_train_w, X_test_w, Y_train_w, Y_test_w = temporal_data_split_forPersistence(weekly_data[col])
    X_train_m, X_test_m, Y_train_m, Y_test_m = temporal_data_split_forPersistence(monthly_data[col])

    res_ps_daily, rmse_ps_daily = persistence_model(X_test_d, Y_test_d)
    res_ps_weekly, rmse_ps_weekly = persistence_model(X_test_w, Y_test_w)
    res_ps_monthly, rmse_ps_monthly = persistence_model(X_test_m, Y_test_m)

    data_daily_persistenceRes.append(res_ps_daily)
    data_weekly_persistenceRes.append(res_ps_weekly)
    data_monthly_persistenceRes.append(res_ps_monthly)

    stats_ps[c] = [col, rmse_ps_daily, rmse_ps_weekly, rmse_ps_monthly]
    
    c+=1

In [ ]:
plot_persistence(data_daily_persistenceRes, daily_data.columns, 'drought_forecasting_persistence_daily', (2, 4))
plot_persistence(data_weekly_persistenceRes, daily_data.columns, 'drought_forecasting_persistence_weekly', (2, 4))
plot_persistence(data_monthly_persistenceRes, daily_data.columns, 'drought_forecasting_persistence_monthly', (2, 4))

In [ ]:
# smoothing
def smooth_sg1(data, frac, p, filename, dimension):

    i, j = dimension
    fig, ax = plt.subplots(i, j, figsize=(35, 25))
    data_filtered = []
    for pos in range(len(data.columns)):
        x = data[data.columns[pos]]
        w = int(len(x)*frac)
        x_filtered = savgol_filter(x, w, p)
        data_filtered.append(x_filtered)

        pos_i = pos//j
        pos_j = pos%j

        ax[pos_i][pos_j].plot(x)
        ax[pos_i][pos_j].plot(x_filtered, color = "green")
        plt.legend(x)
        #plt.xticks(rotation=45)

    
    plt.savefig('plots/' + filename + '.png')
    return data_filtered

In [ ]:
def ses(data, s_level, filename, dimension):

    i, j = dimension
    fig, ax = plt.subplots(i, j, figsize=(35, 25))
    data_filtered = []
    

    for pos in range(len(data.columns)):

        x = data[data.columns[pos]]
        model = SimpleExpSmoothing(x)
        x_filtered = model.fit(smoothing_level=s_level, optimized=False)
        print(x_filtered)
        data_filtered.append(x_filtered.fittedvalues)

        pos_i = pos//j
        pos_j = pos%j

        ax[pos_i][pos_j].plot(x)
        ax[pos_i][pos_j].plot(x_filtered.fittedvalues, color = "green")
        plt.legend(x)
        #plt.xticks(rotation=45)

    
    plt.savefig('plots/' + filename + '.png')
    return data_filtered

In [ ]:
data_smooth_frac001 = smooth_sg1(daily_data, 0.01, 3, 'drought_forecasting_smoothing_frac001_p3',(2, 4))
data_smooth_frac005 = smooth_sg1(daily_data, 0.05, 3, 'drought_forecasting_smoothing_frac005_p3',(2, 4))
data_smooth_frac01 = smooth_sg1(daily_data, 0.1, 3, 'drought_forecasting_smoothing_frac01_p3',(2, 4))

data_smooth_ses_01 = ses(daily_data, 0.1, 'drought_forecasting_smoothing_ses_01', (2,4))
data_smooth_ses_02 = ses(daily_data, 0.2, 'drought_forecasting_smoothing_ses_02', (2,4))
data_smooth_ses_05 = ses(daily_data, 0.5, 'drought_forecasting_smoothing_ses_05', (2,4))

pers_res = pd.DataFrame(columns=['col', 'rmse_sg_0', 'rmse_sg_1', 'rmse_sg_2', 'rmse_ses_0', 'rmse_ses_1', 'rmse_ses_2'])

for col in range(len(daily_data.columns)):

    X_train_0, X_test_0, Y_train_0, Y_test_0 = temporal_data_split_forPersistence(data_smooth_frac001[col])
    X_train_1, X_test_1, Y_train_1, Y_test_1 = temporal_data_split_forPersistence(data_smooth_frac005[col])
    X_train_2, X_test_2, Y_train_2, Y_test_2 = temporal_data_split_forPersistence(data_smooth_frac01[col])

    X_train_3, X_test_3, Y_train_3, Y_test_3 = temporal_data_split_forPersistence(data_smooth_ses_01[col])
    X_train_4, X_test_4, Y_train_4, Y_test_4 = temporal_data_split_forPersistence(data_smooth_ses_02[col])
    X_train_5, X_test_5, Y_train_5, Y_test_5 = temporal_data_split_forPersistence(data_smooth_ses_05[col])


    res_ps_0, rmse_ps_0 = persistence_model(X_test_0, Y_test_0)
    res_ps_1, rmse_ps_1 = persistence_model(X_test_1, Y_test_1)
    res_ps_2, rmse_ps_2 = persistence_model(X_test_2, Y_test_2)

    res_ps_3, rmse_ps_3 = persistence_model(X_test_3, Y_test_3)
    res_ps_4, rmse_ps_4 = persistence_model(X_test_4, Y_test_4)
    res_ps_5, rmse_ps_5 = persistence_model(X_test_5, Y_test_5)


    pers_res.loc[col] = [col, rmse_ps_0, rmse_ps_1, rmse_ps_2, rmse_ps_3, rmse_ps_4, rmse_ps_5]


In [ ]:
pers_res

In [ ]:
daily_data.columns

In [ ]:
# choice: simple exponential smoothing with level = 0.1
data_pos_smoothing = pd.DataFrame(data_smooth_ses_01)
data_pos_smoothing = data_pos_smoothing.transpose()
data_pos_smoothing = data_pos_smoothing.rename(columns = {0:"PRECTOT", 1:"PS", 1:"T2M", 3:"T2MDEW", 4:"T2MWET", 5:"TS", 6:"QV2M"})
data_pos_smoothing

In [ ]:
# differenciation

# one differenciation
data_one_diff = data_pos_smoothing.diff(axis = 0, periods = 1)
# two differenciation
data_two_diff = data_pos_smoothing.diff(axis = 0, periods = 1)
data_two_diff = data_two_diff.diff(axis = 0, periods = 1)

In [ ]:
data_one_diff = data_one_diff.iloc[1:,:]
data_two_diff = data_two_diff.iloc[2:,:]

In [ ]:
res_diff = pd.DataFrame(columns=['col', 'rmse_oneDiff', 'rmse_twoDiff', 'rmse_noDiff'])

for col in range(len(data_one_diff.columns)):

    X_train_oneDiff, X_test_oneDiff, Y_train_oneDiff, Y_test_oneDiff = temporal_data_split_forPersistence(data_one_diff.iloc[:,col])
    X_train_twoDiff, X_test_twoDiff, Y_train_twoDiff, Y_test_twoDiff = temporal_data_split_forPersistence(data_two_diff.iloc[:,col])
    X_train_noDiff, X_test_noDiff, Y_train_noDiff, Y_test_noDiff = temporal_data_split_forPersistence(data_pos_smoothing.iloc[:,col])

    res_oneDiff, rmse_oneDiff = persistence_model(X_test_oneDiff, Y_test_oneDiff)
    res_twoDiff, rmse_twoDiff = persistence_model(X_test_twoDiff, Y_test_twoDiff)
    res_noDiff, rmse_noDiff = persistence_model(X_test_noDiff, Y_test_noDiff)
    
    res_diff.loc[col] = [col, rmse_oneDiff, rmse_twoDiff, rmse_noDiff]

X_train_noDiff, X_test_noDiff, Y_train_noDiff, Y_test_noDiff = temporal_data_split_forPersistence(data_one_diff.iloc[:,col])

In [ ]:
res_diff

In [ ]:
# choice: don't use differenciation
data_pos_smoothing

In [ ]:
data_pos_smoothing.to_csv("intermediate_data/data_drought_forecasting_prepared.csv", encoding='utf-8', columns=data_pos_smoothing.columns, index=True)